# Test code

In [1]:
# Community simulator package
from IPython.display import Image
from community_simulator import *
from community_simulator.usertools import *
from community_simulator.visualization import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends import backend_pdf as bpdf
import numpy as np
import scipy as sp
import pickle
colors = sns.color_palette()
%matplotlib inline

# Community selection package
from community_selection import *
from community_selection.A_experiment_functions import *
from community_selection.B_community_phenotypes import *
from community_selection.C_selection_algorithms import *
from community_selection.D_migration_algorithms import *
from community_selection.usertools import *

In [30]:
# Make dynanmics by default we will use the microbial consumer resource model
def dNdt(N,R,params):
    return MakeConsumerDynamics(assumptions)(N,R,params)
def dRdt(N,R,params):
    return MakeResourceDynamics(assumptions)(N,R,params)
dynamics = [dNdt,dRdt]

# Global parameters
## Default parameters from community-simulator
## !!!Don't touch this dictionary!!!
assumptions = a_default.copy() # Start with default parameters

## Update parameters for community-selection
assumptions.update({
    'SA': 2100 * np.ones(1), #Number of species in each specialist family (here, 3 families of 60 species)
    'MA': 90 * np.ones(1), #Number of resources in each class
    'Sgen': 0, #Number of generalist species (unbiased sampling over alll resource classes)
    "n_wells": 12,
    "m": 0, # Mortality
    "scale": 10**6,  #scale is a conversion factor specifying the number of individual microbial cells present when N = 1.
    "sigma" : 1, # Standard deviation for drawing specifc speices/interaction function
    "alpha": 1, # Scaling factor between species- and interaction-specific function variances
    "l": 0, # Set leakage function to 0 to switch off cross-feeding
    "response": "type III",
    "sigma_max": 5,
    'R0_food': 1000, # Total amount of supplied food
    "rich_medium": True, # Number of food types passed to R0
    "binary_threshold": 1,  
    # The parameters below will be passed to params_simulation
    "n_propagation": 1, # Length of propagation, or hours within a growth cycle
    "n_transfer": 4, # Number of total transfer, or number of passage
    "n_transfer_selection": 2, # Number of transfer implementing seleciton regimes
    "dilution": 1/1000, # Dilution factor at every transfer
    "n_inoc": 10**6,  #Number of cells sampled from the regional species at start
    "n_migration": 10**6, # Number of cells to be migrated in the migration perturbation algorithm
    "R_percent": 0, # Fracion of new resources to be spiked in to the media in the resource perturbation algorithm
    "selected_function": "f1_additive"
})

# Prepare experiment setup in this universe
seed_temp = 1
params, params_simulation = prepare_experiment(assumptions, seed = seed_temp)

In [31]:
data_directory = "data/test/"
list_algorithms = ["simple_screening"]
i = 0
algorithms = make_algorithms(params_simulation)


In [4]:
assumptions = assumptions
params = params
dynamics = dynamics
params_simulation = params_simulation
params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]]
file_name = data_directory + "SP" + str(seed_temp) + "-" + list_algorithms[i]
assembly_type = str(list_algorithms[i])

In [5]:
"""
simple screen
"""
# Set seeds
np.random.seed(2)

# Make initial state
init_state = MakeInitialState(assumptions)

# Make plate
plate = Community(init_state, dynamics, params, scale = assumptions["scale"], parallel = True) 

# Update the community composition by sampling from the pool
print("\nGenerating initial plate")
plate.N = sample_from_pool(plate.N, scale = assumptions["scale"], inocula = params_simulation["n_inoc"])

# Update the supplied resource if assumptions["rich_medium"]
if assumptions["rich_medium"]:
    plate.R = make_rich_medium(plate.R, assumptions)
    plate.R0 = make_rich_medium(plate.R, assumptions) # R0 for refreshing media on passaging if "refresh_resoruce" is turned on

# Add the attributes that are essential to the function measurement to the plate objects 
print("\nAdding attributes that are essential to the community function to the plate object")
plate = add_community_function(plate, dynamics, assumptions, params, params_simulation, params_algorithm)

# Empty list for saving data
plate_data_list = list() # Plate composition
community_function_list = list() # Community function

# Save the inocula composition
plate_data = reshape_plate_data(plate, transfer_loop_index = 0, assembly_type = assembly_type, community_function_name = params_algorithm["community_phenotype"][0]) # Initial state
plate_data_list.append(plate_data)

# Save the initial function
community_function = globals()[params_algorithm["community_phenotype"][0]](plate, assumptions = assumptions) # Community phenotype
richness = np.sum(plate.N >= 1/assumptions["scale"], axis = 0) # Richness
biomass = list(np.sum(plate.N, axis = 0)) # Biomass
function_data = reshape_function_data(community_function_name = params_algorithm["community_phenotype"][0], community_function = community_function, richness = richness, biomass = biomass, transfer_loop_index = 0, assembly_type = assembly_type)        
community_function_list.append(function_data) # Transfer = 0 means that it's before selection regime works upon


# Output the plate composition and community functions if write_composition set True
plate_data.to_csv(file_name + "-" + params_algorithm["community_phenotype"][0] + "-T" + "{:02d}".format(0) + "-composition.txt", index = False)
function_data.to_csv(file_name + "-" + params_algorithm["community_phenotype"][0] + "-T" + "{:02d}".format(0) + "-function.txt", index = False)

print("\nStart propogation")
# Run simulation
for i in range(0, params_simulation["n_transfer"]):
    # Algorithms used in this transfer
    phenotype_algorithm = params_algorithm["community_phenotype"][i]
    selection_algorithm = params_algorithm["selection_algorithm"][i]
    migration_algorithm = params_algorithm["migration_algorithm"][i]

    # Print the propagation progress
    if (i % 5) == 0:
        print("Transfer " + str(i+1))

    # Propagation
    plate.Propagate(params_simulation["n_propagation"])

    # Append the composition to a list
    plate_data = reshape_plate_data(plate, transfer_loop_index = i + 1, assembly_type = assembly_type, community_function_name = phenotype_algorithm) # Transfer = 0 means that it's before selection regime works upon
    plate_data_list.append(plate_data)

    # Community phenotype, richness, and biomass
    community_function = globals()[phenotype_algorithm](plate, assumptions = assumptions) # Community phenotype
    richness = np.sum(plate.N >= 1/assumptions["scale"], axis = 0) # Richness
    biomass = list(np.sum(plate.N, axis = 0)) # Biomass
    function_data = reshape_function_data(community_function_name = phenotype_algorithm, community_function = community_function, richness = richness, biomass = biomass, transfer_loop_index = i + 1 , assembly_type = assembly_type)        
    community_function_list.append(function_data) # Transfer = 0 means that it's before selection regime works upon

    # Output the plate composition and community functions if write_composition set True
    if ((i+1) == assumptions["n_transfer_selection"]) or ((i+1) == assumptions["n_transfer"]):
            plate_data.to_csv(file_name + "-" + phenotype_algorithm + "-T" + "{:02d}".format(i + 1) + "-composition.txt", index = False) # Transfer = 0 means that it's before selection regime works upon
    
    function_data.to_csv(file_name + "-" + phenotype_algorithm + "-T" + "{:02d}".format(i + 1) + "-function.txt", index = False)

    # Save the plate object
    if (i+1) == assumptions["n_transfer_selection"]:
        with open(data_directory + "test.p", "wb") as f:
            pickle.dump({"plate_N":plate.N, "plate_R":plate.R}, f)

    # Passage and tranfer matrix
    transfer_matrix = globals()[selection_algorithm](community_function)
    plate.Passage(transfer_matrix * params_simulation["dilution"])

    # Migration
    m = globals()[migration_algorithm](community_function) 
    plate.N = migrate_from_pool(plate, migration_factor = m, scale = assumptions["scale"], inocula = params_simulation["n_migration"]) # By default, n_migration is the same as n_inoc

print("\nAlgorithm "+ params_algorithm["algorithm_name"][0] + " finished")



Generating initial plate

Adding attributes that are essential to the community function to the plate object

Start propogation
Transfer 1

Algorithm simple_screening finished


In [41]:
list_algorithms = ["migration"]
i = 0
import pickle
plate_screen = pickle.load(open("data/test/test.p", "rb"))

assumptions = assumptions
params = params
dynamics = dynamics
params_simulation = params_simulation
params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]]
file_name = data_directory + "SP" + str(seed_temp) + "-" + list_algorithms[i]
assembly_type = str(list_algorithms[i])
plate_screen["plate_N"]

FileNotFoundError: [Errno 2] No such file or directory: 'data/test/test.p'

In [39]:
"""
Perturbation
"""
# Set seeds
np.random.seed(2)

# Make initial state
init_state = MakeInitialState(assumptions)

# Make plate
plate = Community(init_state, dynamics, params, scale = assumptions["scale"], parallel = True) 

# Update the community composition 
print("\nGenerating initial plate")
plate.N = plate_screen["plate_N"]

# Update the supplied resource if assumptions["rich_medium"]
plate.R = plate_screen["plate_R"]
if assumptions["rich_medium"]:
    plate.R0 = make_rich_medium(plate.R, assumptions) # R0 for refreshing media on passaging if "refresh_resoruce" is turned on

# Add the attributes that are essential to the function measurement to the plate objects 
print("\nAdding attributes that are essential to the community function to the plate object")
plate = add_community_function(plate, dynamics, assumptions, params, params_simulation, params_algorithm)

# Refill the plate with media
transfer_matrix = globals()["no_selection"](community_function)
plate.Passage(transfer_matrix * params_simulation["dilution"])

# Perturbation 
# Migration
migration_algorithm = params_algorithm["migration_algorithm"][params_simulation["n_transfer_selection"]-1]
m = globals()[migration_algorithm](community_function) 
plate.N = migrate_from_pool(plate, migration_factor = m, scale = assumptions["scale"], inocula = params_simulation["n_migration"]) # By default, n_migration is the same as n_inoc



Generating initial plate

Adding attributes that are essential to the community function to the plate object
Migration


In [40]:
plate.isolate_function

AttributeError: 'Community' object has no attribute 'isolate_function'

In [23]:
# Perturbation
if  'knock_in' in params_algorithm["algorithm_name"][0] and selection_algorithm == 'select_top':
    winning_index = np.where(community_function >= np.max(community_function))[0][0] 
    for k in plate.N.columns:
        if k != plate.N.columns[winning_index]:
            s_id = np.random.choice(np.where(plate.N[k]==0)[0])
            if 'isolates' in params_algorithm["algorithm_name"][0]:
                # Knock in an isolate that is 1) not in the resident community 2) have high monoculture function (above 90% isolates in the pool)  
                temp = np.logical_and(np.array(plate.N[k]==0), plate.isolate_function >= np.percentile(plate.isolate_function, q = 90))
                s_id = np.random.choice(np.where(temp)[0]) ## add error check if no isolate does better than the community
            plate.N[k][s_id]= 1/params_simulation["dilution"] * 1/assumptions["scale"]
if 'knock_out' in params_algorithm["algorithm_name"][0] and selection_algorithm == 'select_top':
    winning_index = np.where(community_function >= np.max(community_function))[0][0]
    for k in plate.N.columns:
        if k != plate.N.columns[winning_index]:
            s_id = np.random.choice(np.where(plate.N[k]>0)[0])
            plate.N[k][s_id]=0        
if 'bottleneck' in params_algorithm["algorithm_name"][0]  and selection_algorithm == 'select_top':
    winning_index = np.where(community_function >= np.max(community_function))[0][0] 
    dilution_matrix = np.eye(assumptions['n_wells'])
    dilution_matrix[winning_index,winning_index] = 0
    plate.Passage(np.eye(assumptions['n_wells'])* params_simulation["dilution"])

if 'resource' in params_algorithm["algorithm_name"][0] and selection_algorithm == 'select_top':
    winning_index = np.where(community_function >= np.max(community_function))[0][0]
    #Remove fresh environment that was added by passage
    plate.R = plate.R - plate.R0
    #change default fresh renvironment so that all subsequent rounds use R0
    for k in plate.R0.columns:
        if k != plate.R0.columns[winning_index]: 
            #By default pick 2 resources at randomk
            r_id_remove = np.random.choice(np.where(plate.R0[k]>=0)[0]) 
            r_id_add = np.random.choice(np.where(plate.R0[k]>=0)[0])
            if 'add' in params_algorithm["algorithm_name"][0]: #remove from top and add to random
                r_id_remove = np.where(plate.R0[k]==np.max(plate.R0[k]))[0]
            if 'remove' in params_algorithm["algorithm_name"][0]: #remove from random and add to bottom
                r_id_add = np.where(plate.R0[k]==np.min(plate.R0[k]))[0]
            if 'rescale_add' in params_algorithm["algorithm_name"][0]:  # Increase Fraction of resource
                plate.R0[k][r_id_add] = plate.R0[k][r_id_add]*(1+params_simulation['R_percent']) #increase resource conc by fixed %
            elif 'rescale_remove' in params_algorithm["algorithm_name"][0]: # Decrease Fraction of resource
                plate.R0[k][r_id_remove] = plate.R0[k][r_id_remove]*(1-params_simulation['R_percent'])  #decrease resource 
            elif 'resource_old' in params_algorithm["algorithm_name"][0]:
                plate.R0[k] = plate.R0[k] * (1-params_simulation['R_percent']) #Dilute old resources
                plate.R0[k][r_id_add] = plate.R0[k][r_id_add] + (assumptions['R0_food']*params_simulation['R_percent'])
            else: #Default to resource swap.
                plate.R0[k][r_id_add] = plate.R0[k][r_id_add] + (plate.R0[k][r_id_remove]*params_simulation['R_percent']) #add new resources
                plate.R0[k][r_id_remove] = plate.R0[k][r_id_remove]*(1-params_simulation['R_percent']) #remove new resources
    plate.R0 = plate.R0/np.sum(plate.R0)*assumptions['R0_food'] #Keep this to avoid floating point error and rescale when neeeded.
    #add new fresh environment (so that this round uses R0
    plate.R = plate.R + plate.R0
    
    
if False:
    # Save the inocula composition
    plate_data = reshape_plate_data(plate, transfer_loop_index = 0, assembly_type = assembly_type, community_function_name = params_algorithm["community_phenotype"][0]) # Initial state
    # Save the initial function
    community_function = globals()[params_algorithm["community_phenotype"][0]](plate, assumptions = assumptions) # Community phenotype
    richness = np.sum(plate.N >= 1/assumptions["scale"], axis = 0) # Richness
    biomass = list(np.sum(plate.N, axis = 0)) # Biomass
    function_data = reshape_function_data(community_function_name = params_algorithm["community_phenotype"][0], community_function = community_function, richness = richness, biomass = biomass, transfer_loop_index = 0, assembly_type = assembly_type)        
    # Output the plate composition and community functions if write_composition set True
    plate_data.to_csv(file_name + "-" + params_algorithm["community_phenotype"][0] + "-T" + "{:02d}".format(0) + "-composition.txt", index = False)
    function_data.to_csv(file_name + "-" + params_algorithm["community_phenotype"][0] + "-T" + "{:02d}".format(0) + "-function.txt", index = False)




Generating initial plate

Adding attributes that are essential to the community function to the plate object


In [24]:
plate.N

W0        W1        W2        W3        W4        W5  \
F0 S0     0.000000  0.000000  0.000000  0.000909  0.000000  0.000000   
   S1     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2     0.000000  0.000000  0.000000  0.000013  0.000000  0.000000   
   S3     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S4     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S5     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S6     0.000000  0.000000  0.000000  0.000029  0.000000  0.000000   
   S7     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S8     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S9     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S10    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S11    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S12    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S13    0.000000  0.000000  0.000000  0.000016  0.000000  0.000000   
   S14    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S15    0.000000  0.000000  0.000000  0.000000  0.000000  0.053401   
   S16    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S17    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S18    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S19    0.000000  0.000000  0.000000  0.000190  0.000000  0.000000   
   S20    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S21    0.000018  0.000000  0.000000  0.000000  0.000000  0.000000   
   S22    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S23    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S24    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S25    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S26    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S27    0.000000  0.000000  0.000000  0.000000  0.000004  0.000008   
   S28    0.000000  0.000000  0.000000  0.000000  0.001631  0.000000   
   S29    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...            ...       ...       ...       ...       ...       ...   
   S2070  0.000000  0.000000  0.000000  0.018922  0.000000  0.000000   
   S2071  0.000000  0.000000  0.000004  0.000000  0.000000  0.000000   
   S2072  0.000000  0.000000  0.000000  0.000002  0.000000  0.000000   
   S2073  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2074  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2075  0.000000  0.000000  0.000000  0.000349  0.000000  0.000000   
   S2076  0.000000  0.000000  0.006105  0.000000  0.000000  0.000000   
   S2077  0.000000  0.000000  0.000002  0.000000  0.000000  0.000016   
   S2078  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2079  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2080  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2081  0.000000  0.000000  0.000000  0.000001  0.000000  0.000000   
   S2082  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2083  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2084  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2085  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2086  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2087  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2088  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2089  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2090  0.000000  0.000001  0.000000  0.000000  0.000000  0.000000   
   S2091  0.000000  0.000000  0.000000  0.000023  0.000000  0.000000   
   S2092  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
   S2093  0.000037  0.000000  0.000000  0.000000  0.0000

In [11]:

print("\nStart propogation")
# Run simulation. Starting from the first transfer of stablization
for i in range(params_simulation["n_transfer_selection"]-1, params_simulation["n_transfer"]):
    # Algorithms used in this transfer
    phenotype_algorithm = params_algorithm["community_phenotype"][i]
    selection_algorithm = params_algorithm["selection_algorithm"][i]
    migration_algorithm = params_algorithm["migration_algorithm"][i]

    # Print the propagation progress
    if (i % 5) == 0:
        print("Transfer " + str(i+1))

    # Propagation
    plate.Propagate(params_simulation["n_propagation"])

    # Append the composition to a list
    plate_data = reshape_plate_data(plate, transfer_loop_index = i + 1, assembly_type = assembly_type, community_function_name = phenotype_algorithm) # Transfer = 0 means that it's before selection regime works upon
    plate_data_list.append(plate_data)

    # Community phenotype, richness, and biomass
    community_function = globals()[phenotype_algorithm](plate, assumptions = assumptions) # Community phenotype
    richness = np.sum(plate.N >= 1/assumptions["scale"], axis = 0) # Richness
    biomass = list(np.sum(plate.N, axis = 0)) # Biomass
    function_data = reshape_function_data(community_function_name = phenotype_algorithm, community_function = community_function, richness = richness, biomass = biomass, transfer_loop_index = i + 1 , assembly_type = assembly_type)        
    community_function_list.append(function_data) # Transfer = 0 means that it's before selection regime works upon

    # Output the plate composition and community functions if write_composition set True
    if ((i+1) == assumptions["n_transfer_selection"]) or ((i+1) == assumptions["n_transfer"]):
            plate_data.to_csv(file_name + "-" + phenotype_algorithm + "-T" + "{:02d}".format(i + 1) + "-composition.txt", index = False) # Transfer = 0 means that it's before selection regime works upon
    
    function_data.to_csv(file_name + "-" + phenotype_algorithm + "-T" + "{:02d}".format(i + 1) + "-function.txt", index = False)

    # Save the plate object
    if (i+1) == assumptions["n_transfer_selection"]:
        with open(data_directory + "test.p", "wb") as f:
            pickle.dump({"plate_N":plate.N, "plate_R":plate.R}, f)

    # Passage and tranfer matrix
    transfer_matrix = globals()[selection_algorithm](community_function)
    plate.Passage(transfer_matrix * params_simulation["dilution"])

    # Migration
    m = globals()[migration_algorithm](community_function) 
    plate.N = migrate_from_pool(plate, migration_factor = m, scale = assumptions["scale"], inocula = params_simulation["n_migration"]) # By default, n_migration is the same as n_inoc

print("\nAlgorithm "+ params_algorithm["algorithm_name"][0] + " finished")



Generating initial plate

Adding attributes that are essential to the community function to the plate object

Start propogation
Transfer 1

Algorithm simple_screening finished


In [3]:
data_directory = "data/test/"
list_algorithms = ["simple_screening"]

# Parameters for simulation
params_simulation.update({"selected_function": "f1_additive"}) # selected function

# Make the list of algorithms
algorithms = make_algorithms(params_simulation)

# Simulation
for i in range(len(list_algorithms)):
    simulate_community(
        assumptions = assumptions,
        params = params,
        dynamics = dynamics,
        params_simulation = params_simulation,
        params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]],
        write_composition = True,
        write_function = True,
        file_name = data_directory + "SP" + str(seed_temp) + "-" + list_algorithms[i],
        assembly_type = str(list_algorithms[i]),
    )


Algorithm: simple_screening


 transfer community_phenotype selection_algorithm migration_algorithm
        1         f1_additive        no_selection        no_migration
        2         f1_additive        no_selection        no_migration

Generating initial plate

Adding attributes that are essential to the community function to the plate object

Start propogation
Transfer 1

Algorithm simple_screening finished


In [2]:
x = [1,2,3,4]
y = {"a":1, "b":2}
z = {"x":x, "y":y}
z
     
with open("data/test/test.p", "wb") as f:
    pickle.dump(z, f)
with open("data/test/test.p", "rb") as f:
    print(pickle.load(f))


{'x': [1, 2, 3, 4], 'y': {'a': 1, 'b': 2}}


In [10]:
[print(i) for i in range(5)]
[print(i) for i in range(0, 5)]

0
1
2
3
4
0
1
2
3
4


[None, None, None, None, None]